# データ読込

In [1]:
import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite
from utils.get_holydays import get_japanese_holidays
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
print(f"最小日付: {hol_min}, 最大日付: {hol_max}")
holiday = get_japanese_holidays(hol_min, hol_max)
df.head()

最小日付: 2024-05-01 00:00:00, 最大日付: 2025-06-22 00:00:00


,伝票日付,品名,正味重量,祝日フラグ
0,2024-05-01,混合廃棄物A,1620.0,0
1,2024-05-01,混合廃棄物A,1840.0,0
2,2024-05-01,混合廃棄物A,1730.0,0
3,2024-05-01,混合廃棄物B,2000.0,0
4,2024-05-01,混合廃棄物A,360.0,0


In [2]:
# 品目別合計重量を集計
item_totals = df.groupby("品名")["正味重量"].sum().sort_values(ascending=False)

print(item_totals)


品名
混合廃棄物A          14938790.0
混合廃棄物B           3864960.0
GC 軽鉄･ｽﾁｰﾙ類      2463090.0
選別               1522350.0
木くず              1399580.0
                   ...    
混合廃棄物A(廃ﾌﾟﾗ)          30.0
雑誌                    30.0
選別（水銀灯）               30.0
ﾀｲﾔ                   20.0
混合廃棄物(羽毛)             10.0
Name: 正味重量, Length: 156, dtype: float64


In [3]:
df_reserve = pd.read_csv(
    "/work/app/data/input/yoyaku_data.csv")
df_reserve["予約日"] = pd.to_datetime(df_reserve["予約日"])
df_reserve = df_reserve[df_reserve["予約日"] >= hol_min]
df_reserve = df_reserve[df_reserve["予約日"] <= hol_max]
df_reserve

,予約日,予約得意先名,固定客
0,2025-03-02,泉土木,True
1,2025-03-02,ASMU,False
2,2025-03-02,青木サービス,False
3,2025-03-02,オネストワン,False
4,2025-03-02,おかたずけ田中くん,False
...,...,...,...
4743,2025-05-31,山口興業,True
4744,2025-05-31,谷津商会,True
4745,2025-05-31,吉田機電,False
4746,2025-05-31,ヨシモリ,True


In [4]:
print(df_reserve.columns)

Index(['予約日', '予約得意先名', '固定客'], dtype='object')


### 受入番号用

In [5]:
import os
import glob
import pandas as pd

# ディレクトリ内の全CSVファイルパスを取得
csv_dir = "/work/app/data/input/受入_時刻"
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

# 全CSVを読み込んで結合
dfs = []
for f in csv_files:
    df_tmp = pd.read_csv(f)

    # 伝票日付を整形 → 日付型へ変換
    df_tmp["伝票日付"] = df_tmp["伝票日付"].str.replace(r"\(.*?\)", "", regex=True).str.strip()
    df_tmp["伝票日付"] = pd.to_datetime(df_tmp["伝票日付"], format="%Y/%m/%d")

    # 正味重量をカンマ除去して数値化
    df_tmp["正味重量"] = df_tmp["正味重量"].replace({',': ''}, regex=True).astype(float)

    # 受入番号の欠損を埋めて型変換（念のため）
    df_tmp["受入番号"] = df_tmp["受入番号"].fillna(-1).astype(int)

    dfs.append(df_tmp)

df_Ukeire = pd.concat(dfs, ignore_index=True)

# 必要カラムだけ抽出（品名・重量・受入番号）
df_Ukeire = df_Ukeire[["伝票日付", "品名", "正味重量", "受入番号"]].copy()

# 台数カウント準備（日付・品名ごとの受入番号ユニーク数）
df_count = (
    df_Ukeire.groupby(["伝票日付", "品名"])["受入番号"]
    .nunique()
    .reset_index()
    .rename(columns={"受入番号": "台数"})
)

# （参考）結合しておく場合
df_merged = pd.merge(df_Ukeire, df_count, on=["伝票日付", "品名"], how="left")

# 結果確認
df_merged


,伝票日付,品名,正味重量,受入番号,台数
0,2025-06-01,混合廃棄物A,1110.0,54050,35
1,2025-06-01,混合廃棄物A,120.0,54094,35
2,2025-06-01,混合廃棄物A,270.0,54060,35
3,2025-06-01,混合廃棄物A,1220.0,54044,35
4,2025-06-01,混合廃棄物A,1650.0,54089,35
...,...,...,...,...,...
64680,2024-11-30,混合廃棄物A,50.0,31824,74
64681,2024-11-30,混合廃棄物A,310.0,31822,74
64682,2024-11-30,混合廃棄物A,1140.0,31805,74
64683,2024-11-30,木くず,870.0,31824,9


In [7]:
target_items = ["混合廃棄物A", "混合廃棄物B", "GC 軽鉄･ｽﾁｰﾙ類", "選別", "木くず"]

# 特徴量生成

In [ ]:
from logic.factory_manage.predict_model_v4_4 import full_walkforward_pipeline

pred_df_slim = full_walkforward_pipeline(
    df,
    holidays=holiday,
    target_items=target_items
)


===== 2024-05-10 処理中 =====

===== 2024-05-11 処理中 =====

===== 2024-05-12 処理中 =====

===== 2024-05-13 処理中 =====

===== 2024-05-14 処理中 =====

===== 2024-05-15 処理中 =====

===== 2024-05-16 処理中 =====

===== 2024-05-17 処理中 =====

===== 2024-05-18 処理中 =====

===== 2024-05-19 処理中 =====

===== 2024-05-20 処理中 =====

===== 2024-05-21 処理中 =====

===== 2024-05-22 処理中 =====

===== 2024-05-23 処理中 =====

===== 2024-05-24 処理中 =====

===== 2024-05-25 処理中 =====

===== 2024-05-26 処理中 =====

===== 2024-05-27 処理中 =====

===== 2024-05-28 処理中 =====

===== 2024-05-29 処理中 =====

===== 2024-05-30 処理中 =====

===== 2024-05-31 処理中 =====

===== 2024-06-01 処理中 =====

===== 2024-06-02 処理中 =====

===== 2024-06-03 処理中 =====

===== 2024-06-04 処理中 =====

===== 2024-06-05 処理中 =====

===== 2024-06-06 処理中 =====

===== 2024-06-07 処理中 =====

===== 2024-06-08 処理中 =====

===== 2024-06-09 処理中 =====
  【ステージ2】 合計予測: 63306kg, 実績: 67190kg

===== 2024-06-10 処理中 =====
  【ステージ2】 合計予測: 19292kg, 実績: 0kg

===== 2024-06-11 処理中 =====
  【ステー

/work/app/logic/factory_manage/predict_model_v4_4.py:112: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [8]:
from logic.factory_manage.predict_model_v4_4 import full_weekly_simulation
weekly_df = full_weekly_simulation(
    df,
    target_items=target_items
)

===== 2024-06-04 処理中 =====


KeyError: "['曜日', '週番号'] not in index"